### Determinando un FVG
Lo primero que debemos hacer es leer data.
Vamos a descargar data de BTCUSDT en el rango 2023-08-09 - 2023-08-11 (9 al 11 de Agosto de 2023) en la temporalidad M5

In [1]:
import pandas as pd
import yfinance as yf
from finta import TA

In [2]:
# Leyendo data
symbols = 'BTCUSDT'
start_date = '2023-08-09'